In [1]:
import yfinance as yf

In [2]:
import numpy as np

In [3]:
import pandas as pd

In [4]:
import statsmodels.api as sm

In [5]:
import datetime
from dateutil.relativedelta import relativedelta
from datetime import date

In [6]:
stock = 'smci'

cutoff=0.52
invest = 100
years_of_data_to_process = 25

In [7]:
currentDateTime = datetime.datetime.now()
date = currentDateTime.date()
year = date.strftime("%Y")
start_year =  int(year) - years_of_data_to_process
start = datetime.datetime(start_year, 1, 1)

In [8]:
data =  yf.download(stock, start=start)

[*********************100%***********************]  1 of 1 completed


In [9]:
data

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2007-03-29,8.850000,9.310000,8.660000,8.760000,8.760000,6079800
2007-03-30,8.830000,9.000000,8.550000,8.990000,8.990000,419500
2007-04-02,9.000000,10.590000,8.990000,10.380000,10.380000,749100
2007-04-03,10.700000,11.660000,10.400000,10.580000,10.580000,835700
2007-04-04,10.400000,10.990000,10.030000,10.160000,10.160000,309500
...,...,...,...,...,...,...
2021-06-02,35.349998,36.389999,34.840000,36.279999,36.279999,456400
2021-06-03,36.220001,36.450001,35.770000,36.410000,36.410000,108000
2021-06-04,36.439999,36.770000,36.150002,36.689999,36.689999,97300


In [10]:
df = data["Close"].pct_change() * 100

In [11]:
df = df.rename("Today_Change_%")
df = df.reset_index()

In [12]:
df1 = pd.merge(data,df, on="Date")
#df1

In [13]:
df1.insert(7,'Volume_Lag', None)
df1['Volume_Lag'] = df1.Volume.shift(1).values/1000000000
df1.Volume = df1.Volume/1000000000

In [14]:
df1["Trend"] =  (df1["Close"] - df1["Low"])/ ((df1["High"] - df1["Low"]))

In [15]:
df1["Trend_Lag"] = df1["Trend"].shift(1)

In [16]:
df1['Close-Open'] =  df1['Close'] - df1['Open']
df1['Close-Open_Lag'] = df1['Close-Open'].shift(1)

In [17]:
df1['High-Low'] =  df1['High'] - df1['Low']
df1['High-Low_Lag'] = df1['High-Low'].shift(1)

In [18]:
df1 = sm.add_constant(df1)
df1["Up_Down"] = [1 if (i > 0) else 0 for i in df1["Today_Change_%"]]

In [19]:
df1.dropna(inplace= True)

In [20]:
df1['SMA20'] = df1['Close'].ewm(span=20, adjust=False).mean()
df1.dropna(inplace= True)
#df1

In [21]:
df1['SMA50'] = df1['Close'].ewm(span=50, adjust=False).mean()
df1.dropna(inplace= True)
#df1

In [22]:
df1['SMA20-SMA50'] = df1.SMA20 - df1.SMA50
df1['SMA20-SMA50_Lag'] = df1['SMA20-SMA50'].shift(1)
df1

,const,Date,Open,High,Low,Close,Adj Close,Volume,Volume_Lag,Today_Change_%,...,Trend_Lag,Close-Open,Close-Open_Lag,High-Low,High-Low_Lag,Up_Down,SMA20,SMA50,SMA20-SMA50,SMA20-SMA50_Lag
1,1.0,2007-03-30,8.830000,9.000000,8.550000,8.990000,8.990000,0.000420,0.006080,2.625565,...,0.153847,0.160000,-0.090000,0.450000,0.650001,1,8.990000,8.990000,0.000000,NaN
2,1.0,2007-04-02,9.000000,10.590000,8.990000,10.380000,10.380000,0.000749,0.000420,15.461628,...,0.977777,1.380000,0.160000,1.600000,0.450000,1,9.122381,9.044510,0.077871,0.000000
3,1.0,2007-04-03,10.700000,11.660000,10.400000,10.580000,10.580000,0.000836,0.000749,1.926780,...,0.868750,-0.120000,1.380000,1.260000,1.600000,1,9.261202,9.104725,0.156477,0.077871
4,1.0,2007-04-04,10.400000,10.990000,10.030000,10.160000,10.160000,0.000309,0.000836,-3.969755,...,0.142857,-0.240000,-0.120000,0.960000,1.260000,0,9.346801,9.146108,0.200693,0.156477
5,1.0,2007-04-05,10.020000,10.250000,9.520000,9.800000,9.800000,0.000425,0.000309,-3.543304,...,0.135417,-0.220000,-0.240000,0.730000,0.960000,0,9.389963,9.171751,0.218212,0.200693
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3569,1.0,2021-06-02,35.349998,36.389999,34.840000,36.279999,36.279999,0.000456,0.000195,3.038909,...,0.662161,0.930000,0.129997,1.549999,0.739998,1,35.925109,36.231447,-0.306338,-0.341713
3570,1.0,2021-06-03,36.220001,36.450001,35.770000,36.410000,36.410000,0.000108,0.000456,0.358327,...,0.929032,0.189999,0.930000,0.680000,1.549999,1,35.971289,36.238449,-0.267160,-0.306338
3571,1.0,2021-06-04,36.439999,36.770000,36.150002,36.689999,36.689999,0.000097,0.000108,0.769016,...,0.941175,0.250000,0.189999,0.619999,0.680000,1,36.039737,36.256157,-0.216420,-0.267160
3572,1.0,2021-06-07,36.790001,37.150002,36.470001,37.000000,37.000000,0.000167,0.000097,0.844921,...,0.870965,0.209999,0.250000,0.680000,0.619999,1,36.131191,36.285327,-0.154136,-0.216420


In [23]:
#crossovers = pd.DataFrame()
#crossovers['Dates'] = SMA20['Date']
#crossovers['Price'] = [i for i in df1.Close]
#crossovers['SMA20'] = SMA20['Close']
#crossovers['SMA50'] = SMA50['Close']
#crossovers

In [24]:
df1.dropna(inplace= True)
#df1

In [25]:
X = df1[['const','Trend_Lag','SMA20-SMA50_Lag','Close-Open_Lag','High-Low_Lag','Volume_Lag']]

In [26]:
y = df1["Up_Down"].values

In [27]:
model = sm.Logit(y,X)

In [28]:
result =  model.fit()

Optimization terminated successfully.
         Current function value: 0.692216
         Iterations 5


In [29]:
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                      y   No. Observations:                 3570
Model:                          Logit   Df Residuals:                     3564
Method:                           MLE   Df Model:                            5
Date:                Tue, 08 Jun 2021   Pseudo R-squ.:               0.0007778
Time:                        17:27:06   Log-Likelihood:                -2471.2
converged:                       True   LL-Null:                       -2473.1
Covariance Type:            nonrobust   LLR p-value:                    0.5716
===================================================================================
                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
const               0.0848      0.092      0.917      0.359      -0.096       0.266
Trend_Lag          -0.1218      0.144     -0.847      0.397      -0.404       0.160
SMA20-SMA50_Lag    -0.0546      0.037     -1.474      0.140      -0.127       0.018
Close-Open_Lag      0.0237      0.089      0.266      0.790      -0.151       0.199
High-Low_Lag        0.0629      0.078      0.811      0.417      -0.089       0.215
Volume_Lag         -0.8990     62.850     -0.014      0.989    -124.084     122.286
===================================================================================
"""

In [30]:
prediction = result.predict(X)

In [31]:
df1['Prediction_Caculated'] = pd.array(prediction)
df1['Prediction_indicator'] = pd.array([1 if i > cutoff else 0 for i in prediction])
#df1.tail(10)

In [32]:
y = df1["Up_Down"].values

In [33]:
def confusion_matrix(act,pred):
    predtrans = ['Up' if i > cutoff else 'Down' for i in pred]
    actuals = ['Up' if i > 0 else 'Down' for i in act]
    confusion_matrix = pd.crosstab(pd.Series(actuals),
                                   pd.Series(predtrans),
                                   rownames = ["Actual"],
                                   colnames = ["Predict"]
                                  )
    return confusion_matrix

In [34]:
confusion_matrix(y,prediction)

Predict,Down,Up
Actual,,
Down,1175,560
Up,1197,638


In [35]:
z = confusion_matrix(y,prediction)
try:
    print((z.loc['Down','Down'] + z.loc['Up','Up']) / len(df1))
except:
    pass

0.5078431372549019


In [36]:
try:
    print( (z.loc['Down', 'Down']+ z.loc['Up','Up']) / (z.loc['Down', 'Down']+ z.loc['Up','Up'] + z.loc['Down','Up']) )
except:
    pass

0.7640117994100295


In [37]:
df1 = df1.assign(share=np.nan,money=np.nan)
#df1

In [38]:
diff_years = round((df1.iloc[-1,1] - df1.iloc[0,1])/np.timedelta64(1,'Y') + 0.5)   

In [39]:
#Simulate Investment transaction buy on opening when predict UP and sell daily average when predict DOWN

def buy_sell(open_price, sell_price,prediction, money, share):
    if prediction == 1 and money != 0:
        share =  money / open_price
        money = 0
    elif prediction == 0 and share != 0:
        money = share * sell_price
        share = 0
    else: pass
    return [money, share]
money = invest
share = 0
for i in range(len(df1)):
    [money, share] = buy_sell(df1.iloc[i,2],(df1.iloc[i,3]+df1.iloc[i,4])/2,df1.iloc[i,22], money, share)
    df1.iloc[i,23] = share
    df1.iloc[i,24] = money
    

In [40]:
#df1

In [41]:
print("\nIf ${:,.2f} was invested in [ {} ], and Just Hold and Not Trade for {:,} years, the ROI = ${:,.2f}".format( invest, stock.upper(), diff_years, invest/data.iloc[0, 0] * data.iloc[-1,0]))


If $100.00 was invested in [ SMCI ], and Just Hold and Not Trade for 15 years, the ROI = $420.34


In [42]:
print ("\nIf ${:,.2f} was invested {:2d} years ago, buy and sell according this script\'s recommandation, the ROI = ${:,.2f}".format(invest, diff_years, (money + (share * df1.iloc[-1,6]))))


If $100.00 was invested 15 years ago, buy and sell according this script's recommandation, the ROI = $342.33


In [43]:
df1_summary=df1[['Date', 'Up_Down','Prediction_indicator']].copy()
df1_summary['Stock Market Performance'] = df1_summary['Up_Down'].apply(lambda x: 'Up' if x > 0 else 'Down')
df1_summary['Scribe Predection'] = df1_summary['Prediction_indicator'].apply(lambda x: 'Up' if x > 0 else 'Down')
print (df1_summary[['Date','Stock Market Performance','Scribe Predection']].tail(15))

           Date Stock Market Performance Scribe Predection
3559 2021-05-18                     Down              Down
3560 2021-05-19                     Down                Up
3561 2021-05-20                       Up              Down
3562 2021-05-21                     Down              Down
3563 2021-05-24                       Up                Up
3564 2021-05-25                     Down              Down
3565 2021-05-26                       Up                Up
3566 2021-05-27                       Up              Down
3567 2021-05-28                     Down                Up
3568 2021-06-01                       Up                Up
3569 2021-06-02                       Up              Down
3570 2021-06-03                       Up                Up
3571 2021-06-04                       Up              Down
3572 2021-06-07                       Up              Down
3573 2021-06-08                     Down              Down


In [44]:
#df1

In [45]:
#print(df1.iloc[:,[7,10,11,12,13,14,15,16,17,18,19,20,21,22]])

In [46]:
print ("\nToday [ %s ] actually went up," %stock.upper(), end = ' ') if (df1.iloc[-1,16] == 1) else print ("\nToday [ %s ] actually went down," %stock.upper(), end = " ")
print ("--- base on yesterday\'s data, ", end = '')
print ("We Predication [ %s ] should be going up." %stock.upper()) if (df1.iloc[-1,22] == 1) else print ("We Predicae [ %s ] should be going down." %stock.upper())
print ("\n=========> Actual and Predication MATCH <=========") if (df1.iloc[-1,16] == df1.iloc[-1,22]) else print("\n=========> Actual and Predication DO NOT match <=========")


Today [ SMCI ] actually went down, --- base on yesterday's data, We Predicae [ SMCI ] should be going down.

=========> Actual and Predication MATCH <=========


In [47]:
x_tran= df1[df1.Date.dt.year < 2021][['const','Trend_Lag','SMA20-SMA50_Lag','Close-Open_Lag','High-Low_Lag','Volume_Lag']]
y_train=df1[df1.Date.dt.year < 2021]["Up_Down"]
x_test= df1[df1.Date.dt.year >= 2021][['const','Trend_Lag','SMA20-SMA50_Lag','Close-Open_Lag','High-Low_Lag','Volume_Lag']]
y_test= df1[df1.Date.dt.year >= 2021]["Up_Down"]

In [48]:
model = sm.Logit(y_train,x_tran)
result=model.fit()

Optimization terminated successfully.
         Current function value: 0.692211
         Iterations 5


In [49]:
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                Up_Down   No. Observations:                 3462
Model:                          Logit   Df Residuals:                     3456
Method:                           MLE   Df Model:                            5
Date:                Tue, 08 Jun 2021   Pseudo R-squ.:               0.0008848
Time:                        17:27:08   Log-Likelihood:                -2396.4
converged:                       True   LL-Null:                       -2398.6
Covariance Type:            nonrobust   LLR p-value:                    0.5148
===================================================================================
                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
const               0.0873      0.093      0.937      0.349      -0.095       0.270
Trend_Lag          -0.1313      0.145     -0.904      0.366      -0.416       0.153
SMA20-SMA50_Lag    -0.0583      0.038     -1.534      0.125      -0.133       0.016
Close-Open_Lag      0.0163      0.091      0.179      0.858      -0.162       0.195
High-Low_Lag        0.0578      0.079      0.727      0.467      -0.098       0.214
Volume_Lag         -0.1279     63.206     -0.002      0.998    -124.009     123.754
===================================================================================
"""

In [50]:
prediction = result.predict(x_test)
confusion_matrix(y_test, prediction)

Predict,Down,Up
Actual,,
Down,41,7
Up,52,8


In [51]:
z = confusion_matrix(y_test,prediction)

In [52]:
try:
    print ("\n=========> Prediction Accuracy Rate: %.4f <=========\n"  %((z.loc['Down','Down'] + z.loc['Up','Up']) / len(x_test)))
except:
    print ("\n=========> Predication effectiveness is not avairable <=========\n" )


=========> Prediction Accuracy Rate: 0.4537 <=========



In [53]:
prediction = result.predict(x_test)
now_up_down  = result.predict([1.0, df1.iloc[-1, 10], df1.iloc[-1, 19], df1.iloc[-1, 12], df1.iloc[-1, 14], df1.iloc[-1, 7]])
print ("\n=========> Current trend = %.4f,  " %now_up_down, end=' ')
print ("[ %s ] will go up! <=========" %stock.upper()) if now_up_down > cutoff else print ("[ %s ] will go down! <=========" %stock.upper()) 


=========> Current trend = 0.5274,   [ SMCI ] will go up! <=========


In [54]:
#df1.to_csv('fb.csv', index = False)